In [ ]:
import matplotlib.pyplot as plt
from   matplotlib.colors import LogNorm
import numpy as np
from   scipy.stats import norm


In [ ]:
data = []
f = open("data/Synthetic_Sin+anom/varNoise_fixedNumberAnomaliesfixedLength/SinusRW_Length_112000_AnomalyL_200_AnomalyN_60_NoisePerc_5.ts", "r")
data = f.read()
data = (data.rstrip().split('\n')) 
data = [float(i) for i in data]
f.close()
cpts = []
f = open("data/Synthetic_Sin+anom/varNoise_fixedNumberAnomaliesfixedLength/SinusRW_Length_112000_AnomalyL_200_AnomalyN_60_NoisePerc_5_Annotations.txt", "r")
cpts = f.read()
cpts = (cpts.rstrip().split('\n')) 
cpts = [int(i) for i in cpts]
f.close()

In [ ]:
# Read in NAB data
f = open("data/realAWSCloudwatch/ec2_cpu_utilization_825cc2.csv")
data = f.read()
print(data)
#print(data.shape)

In [ ]:
data = data[:1000]

In [ ]:
cpts = cpts[:1]

In [ ]:
cpts

In [ ]:
type(data)

In [ ]:
%matplotlib inline
import numpy as np
np.random.seed(555)
import matplotlib.pyplot as plt
import bocd
import csv


In [ ]:
#f = open("Synthetic_Sin+anom/varNoise_fixedNumberAnomaliesfixedLength/SinusRW_Length_112000_AnomalyL_200_AnomalyN_60_NoisePerc_5.ts", "r")
#f = open("RealDatasets/MBA_ECG_801.ts", "r")
#f = open("lga.csv", "r")
test_signal = []
with open('data/lga.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        if row[0] == 'LGA':
            test_signal.append(row)
        
# test_signal = []
# data = f.readlines()
# for d in data:
#     test_signal.append(d)
test_signal = np.asarray(test_signal)
print(len(test_signal))
print(test_signal[0])
test_signal = test_signal[:,-1]
test_signal = test_signal[1:]
test_signal = test_signal.astype(np.float)
test_signal = np.flip(test_signal)

test_signal = (test_signal - np.mean(test_signal)) / np.std(test_signal)
print(test_signal[1])


print(test_signal.shape)

# test_signal = test_signal[1200:1400]
# f.close()

In [ ]:
import json

with open('data/gdp_argentina.json') as f:
  data = json.load(f)

test_signal = data['series'][0]['raw']
test_signal = np.asarray(test_signal)
test_signal = test_signal.astype(np.float)

test_signal = (test_signal - np.mean(test_signal)) / np.std(test_signal)
print(test_signal)

In [ ]:
#print(test_signal[:100])
# Generate test data
# test_signal = np.concatenate(
#     [np.random.normal(0.1, 1.0, 300), 
#      np.random.normal(2.0, 0.8, 300),
#      np.random.normal(1.0, 0.5, 300),
#      np.random.normal(3.0, 1.0, 300)])
plt.plot(test_signal)

In [ ]:
# Initialize object
bc = BayesianOnlineChangePointDetection(ConstantHazard(200), StudentT(mu=0, kappa=1, alpha=1, beta=1))

# Online estimation and get the maximum likelihood r_t at each time point
rt_mle = np.empty(test_signal.shape)
for i, d in enumerate(test_signal):
    if i >=0:
        #print(i)
        bc.update(test_signal[i-0:i+1])
        rt_mle[i] = bc.rt
        
        #bc.update(test_signal[i])
        #rt_mle[i] = bc.rt
    else:
        bc.update(d)
        rt_mle[i] = 0
    
    
    
    

In [ ]:
alpha = [0.01,1,100]
beta = [0.01,1,100]
kappa = [0.01,1,100]
L=0

for a in alpha:
    for b in beta:
        for k in kappa:
            # Initialize object
            bc = BayesianOnlineChangePointDetection(ConstantHazard(50), StudentT(mu=0, kappa=k, alpha=a, beta=b))



            # Online estimation and get the maximum likelihood r_t at each time point
            rt_mle = np.empty(test_signal.shape)
            for i, d in enumerate(test_signal):
                if i >=L:
                    #print(i)
                    bc.update(test_signal[i-L:i+1])
                    #print(bc.rt)
                    rt_mle[i] = bc.rt

                    #bc.update(test_signal[i])
                    #rt_mle[i] = bc.rt
                else:
                    bc.update(d)
                    rt_mle[i] = 0
                    
            print("alpha = ", a, " Beta = ", b, " kappa = ", k)
            plt.plot(rt_mle)
            plt.show()
            # Plot data with estimated change points
            plt.plot(test_signal, alpha=0.5, label="observation")
            index_changes = np.where(np.diff(rt_mle)<-1)[0]
            plt.scatter(index_changes, test_signal[index_changes], c='green', label="change point")
            plt.show()




In [ ]:
plt.show()
plt.plot(rt_mle)
plt.show()

In [ ]:
# Plot data with estimated change points
plt.plot(test_signal, alpha=0.5, label="observation")
index_changes = np.where(np.diff(rt_mle)<-1)[0]
plt.scatter(index_changes, test_signal[index_changes], c='green', label="change point")

In [ ]:
import numpy as np


class BayesianOnlineChangePointDetection:
    def __init__(self, hazard, distribution):
        self.hazard = hazard
        self.distribution = distribution
        self.T = 0
        self.beliefs = np.zeros((1, 2),dtype=np.float64)
        self.beliefs[0, 0] = 1.0

    def reset_params(self):
        self.T = 0
        self.beliefs = np.zeros((1, 2))
        self.beliefs[0, 0] = 1.0

    def _expand_belief_matrix(self):
        rows = np.zeros((1, 2),dtype=np.float64)
        self.beliefs = np.concatenate((self.beliefs, rows), axis=0)

    def _shift_belief_matrix(self):
        self.beliefs[:, 0] = self.beliefs[:, 1]
        self.beliefs[:, 1] = 0.0

    def update(self, x):
        
        
        # Evaluate Predictive Probability (3 in Algortihm 1)
        if type(x) is not np.ndarray:
            self._expand_belief_matrix()

            self.distribution.update_params(x)
            # Update internal state
            #print(len(self.beliefs))

            self.T += 1
        else:
            self._expand_belief_matrix()

            pi_t = self.distribution.pdf(x)
            # Calculate H(r_{t-1})
            h = self.hazard(self.rt)

            # Calculate Growth Probability (4 in Algorithm 1)
            #print(pi_t)
            #print(len(self.beliefs))
            #print(self.beliefs)
            self.beliefs[1 : self.T + 2, 1] = self.beliefs[: self.T + 1, 0] * pi_t * (1 - h)

            # Calculate Changepoint Probabilities (5 in Algorithm 1)
            self.beliefs[0, 1] = (self.beliefs[: self.T + 1, 0] * pi_t * h).sum()

            # Determine Run length Distribution (7 in Algorithm 1)
            self.beliefs[:, 1] = self.beliefs[:, 1] / self.beliefs[:, 1].sum()

            # Update sufficient statistics (8 in Algorithm 8)
            self.distribution.update_params(x[-1])

            # Update internal state
            self._shift_belief_matrix()
            self.T += 1
            
            
        
        
    @property
    def rt(self):
        return np.where(self.beliefs[:, 0] == self.beliefs[:, 0].max())[0]

    @property
    def belief(self):
        return self.beliefs[:, 0]


In [ ]:
import numpy as np


class Hazard:
    def __call__(self, *args, **kwargs):
        raise NotImplementedError()


class ConstantHazard(Hazard):
    def __init__(self, _lambda):
        self._lambda = _lambda

    def __call__(self, r):
        """
        Args:
          r: The length of the current run (np.ndarray or scalar)

        Returns:
          p: Changepoint Probabilities(np.ndarray with shape = r.shape)
        """
        if isinstance(r, np.ndarray):
            shape = r.shape
        else:
            shape = 1

        return np.ones(shape) / self._lambda


In [ ]:
import numpy as np
from scipy import stats


class Distribution:
    def reset_params(self):
        raise NotImplementedError()

    def pdf(self, x):
        raise NotImplementedError()

    def update_params(self, x):
        raise NotImplementedError()


class StudentT(Distribution):
    """ Generalized Student t distribution 
    https://en.wikipedia.org/wiki/Student%27s_t-distribution#Generalized_Student's_t-distribution

    This setting corresponds to select
      1: Gaussian distribution as a likelihood
      2: normal-Gamma distribution as a prior for Gaussian
    """

    def __init__(self, mu=0, kappa=1, alpha=1, beta=1):
        self.mu0 = np.array([mu],dtype=np.float64)
        self.kappa0 = np.array([kappa],dtype=np.float64)
        self.alpha0 = np.array([alpha],dtype=np.float64)
        self.beta0 = np.array([beta],dtype=np.float64)
        # We need the following lines to prevent "outside defined warning"
        self.muT = self.mu0.copy()
        self.kappaT = self.kappa0.copy()
        self.alphaT = self.alpha0.copy()
        self.betaT = self.beta0.copy()

    def reset_params(self):
        self.muT = self.mu0.copy()
        self.kappaT = self.kappa0.copy()
        self.alphaT = self.alpha0.copy()
        self.betaT = self.beta0.copy()

    def pdf(self, x):
        """ Probability Density Function
        """
        #print(x)
        #print()
        #return stats.t.pdf(x, loc=self.muT, df=2 * self.alphaT, scale=np.sqrt(self.betaT * (self.kappaT + 1) / (self.alphaT * self.kappaT)),)
        probs = []
        for xd in x:
            probs.append(stats.t.pdf(xd, loc=self.muT, df=2 * self.alphaT, scale=np.sqrt(self.betaT * (self.kappaT + 1) / (self.alphaT * self.kappaT)),))
        
        ans = np.zeros(len(probs[0]),dtype=np.float64)
        for pro in probs:
            for i in range(len(pro)):
                ans[i] += pro[i]
        ans /= len(probs[0])
        #print(ans)
        return ans
        #return stats.t.pdf(x, loc=self.muT, df=2 * self.alphaT, scale=np.sqrt(self.betaT * (self.kappaT + 1) / (self.alphaT * self.kappaT)),)

    def update_params(self, x):
        """Update Sufficient Statistcs (Parameters)

        To understand why we use this, see e.g.
        Conjugate Bayesian analysis of the Gaussian distribution, Kevin P. Murphy∗
        https://www.cs.ubc.ca/~murphyk/Papers/bayesGauss.pdf
        3.5 Posterior predictive
        """
        self.betaT = np.concatenate(
            [
                self.beta0,
                (self.kappaT + (self.kappaT * (x - self.muT) ** 2) / (2 * (self.kappaT + 1))),
            ]
        )
        self.muT = np.concatenate([self.mu0, (self.kappaT * self.muT + x) / (self.kappaT + 1)])
        self.kappaT = np.concatenate([self.kappa0, self.kappaT + 1])
        self.alphaT = np.concatenate([self.alpha0, self.alphaT + 0.5])
